# ASBE - Automatic Stopping for Batch Experiments

> API details.

In [ ]:
#hide
from nbdev import *

In [ ]:
%nbdev_default_export core

Cells will be exported to asbe.core,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%nbdev_export
import numpy as np

from modAL.models.base import BaseLearner
from sklearn.base import BaseEstimator, ClassifierMixin, RegressorMixin
from typing import Union, Optional
from copy import deepcopy
from pylift.eval import UpliftEval

In [ ]:
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

In [ ]:
%nbdev_export
def random_batch_sampling(classifier, X_pool, n2):
    "Randomly sample a batch from a pool of unlabaled samples"
    n_samples = X_pool.shape[0]
    query_idx = np.random.choice(range(n_samples), size=n2,replace=False)
    return X_pool[query_idx], query_idx

def uncertainty_batch_sampling(classifier, X_pool, n2, **kwargs):
    "Select the top $n_2$ most uncertain units"
    ite_preds, y1_preds, y_preds = classifier.predict(X_pool, **kwargs)
    # Calculate variance based on predicted
    if y1_preds.shape[0] <= 1 or \
    len(y1_preds.shape) <= 1:
            raise Exception("Not possible to calculate uncertainty when dimensions <=1 ")
    ite_vars = np.var(classifier.estimator.y1_preds - classifier.estimator.y0_preds, axis=1)
    query_idx = np.argsort(ite_vars)[-n2:][::-1]
        
    return X_pool[query_idx], query_idx

def expected_model_change_maximization(classifier, X_pool, n2, **kwargs):
    # Get mean of predicted ITE
    ite_train_preds, y1_train_preds, y_train_preds = \
        classifier.predict(classifier.X_training, **kwargs)
    classifier.approx_model = classifier.approx_model.fit(
        classifier.X_training,
        ite_train_preds if ite_train_preds.shape[1]>0 else np.mean(ite_train_preds, axis=1))
    print(classifier.approx_model)

In [ ]:
%nbdev_export
estimator_type = ClassifierMixin
class ASLearner(BaseLearner):
    """A(ctively)S(topping)Learner class for automatic stopping in batch-mode AL"""
    def __init__(self,
                 estimator: estimator_type=None, 
                 query_strategy=None,
                 assignment_fc=None,
                 X_training: np.ndarray = None,
                 t_training: np.ndarray = None,
                 y_training: np.ndarray = None,
                 X_pool: np.ndarray = None,
                 X_test: np.ndarray = None,
                 approx_model: RegressorMixin = None
                ) -> None:
        self.estimator = estimator
        self.query_strategy = query_strategy
        self.assignment_fc = assignment_fc
        self.X_training = X_training
        self.y_training = y_training
        self.t_training = t_training
        self.X_pool     = X_pool
        self.X_test     = X_test
        self.approx_model = approx_model
        
    def _add_queried_data_class(self, X, t, y):
        self.X_training = np.vstack((self.X_training, X))
        self.t_training = np.concatenate((self.t_training, t))
        self.y_training = np.concatenate((self.y_training, y))
    
    def _update_estimator_values(self):
        self.estimator.__dict__.update(X_training = self.X_training,
                               y_training  =        self.y_training,
                               t_training  =        self.t_training,
                               X_test      =        self.X_test)

    def teach(self, X_new, t_new, y_new):
        """Teaching new instances to the estimator selected bu the query_strategy
        
        If no `assignment_fc` is added, all selected samples are used
        If assignment function is added, only those instances are used, where
        $\hat{T} = T$
        """
        if self.assignment_fc is None:
            self._add_queried_data_class(X_new, t_new, y_new)
            self.fit()
    
    def fit(self):
        self._update_estimator_values()
        self.estimator.fit()
        
    def predict(self, X=None, **kwargs):
        """Method for predicting treatment effects within Active Learning
        
        Default is to predict on the unlabeled pool"""
        if X is None:
            raise Exception("You need to supply an unlabeled pool of instances (with shape (-1,{}))".format(self.X_training.shape[1]))
        self.preds = self.estimator.predict(X, **kwargs)
        return self.preds
    
    def score(self, preds=None, y_true=None, t_true=None, metric = "Qini"):
        """
        Scoring the predictions - either ITE or observed outcomes are needed.
        
        If observed outcomes are provided, the accompanying treatments are also needed.
        """
        if metric == "Qini":
            upev = UpliftEval(t_true, y_true, self.preds[0] if preds is None else preds)
            self.scores = upev
        return self.scores.q1_aqini

In [ ]:
%nbdev_export
class ITEEstimator(BaseEstimator):
    """ Class for building a naive estimator for ITE estimation
    """
    def __init__(self,
                 model: estimator_type = None,
                 two_model: bool = False,
                 **kwargs
                ) -> None:
        self.model = model
        self.two_model = two_model

    def fit(self,X_training: np.ndarray = None,
                 t_training: np.ndarray = None,
                 y_training: np.ndarray = None,
                 X_test: np.ndarray = None):
        if X_training is not None:
            self.X_training = X_training
            self.y_training = y_training
            self.t_training = t_training
            self.X_test = X_test
        self.N_training = self.X_training.shape[0]
        # if "N_training" not in self.__dict__:
        #     self.N_training = self.X_training.shape[0]
        if self.two_model:
            self.m1 = deepcopy(self.model)
            control_ix = np.where(self.t_training == 0)[0]
            self.model.fit(self.X_training[control_ix,:],
                           self.y_training[control_ix])
            self.m1.fit(self.X_training[-control_ix,:],
                        self.y_training[-control_ix])
        else:
            self.model.fit(np.hstack((self.X_training,
                                      self.t_training.reshape((self.N_training, -1)))),
                           self.y_training)
            
    def _predict_without_proba(self, model, X, **kwargs):
        return model.predict(X,
            return_mean = kwargs["return_mean"] if "return_mean" in kwargs else True)
            
    def predict(self, X=None, **kwargs):
        if X is None:
            X = self.X_test
        N_test = X.shape[0]
        try:
            if self.two_model:
                self.y1_preds = self.m1.predict_proba(X)[:,1]
                self.y0_preds = self.model.predict_proba(X)[:,1]
            else:
                
                self.y1_preds = self.model.predict_proba(
                                    np.hstack((X,
                                    np.ones(N_test).reshape(-1,1))))[:,1]
                self.y0_preds = self.model.predict_proba(
                    np.hstack((X,
                               np.zeros(N_test).reshape(-1,1))))[:,1]
        except AttributeError:
            if type(self.model) is XBART:
                if self.two_model:
                    self.y1_preds = self._predict_without_proba(self.m1, X, **kwargs)
                    self.y0_preds = self._predict_without_proba(self.model, X, **kwargs)
                else: 
                    self.y1_preds = self._predict_without_proba(self.model,
                             np.hstack((X,
                             np.ones(N_test).reshape(-1,1))), **kwargs)
                    self.y0_preds = self._predict_without_proba(self.model,
                        np.hstack((X,
                                   np.zeros(N_test).reshape(-1,1))), **kwargs)
        return self.y1_preds - self.y0_preds, self.y1_preds, self.y0_preds

In [ ]:
X = np.random.normal(size = 1000).reshape((500,2))
t = np.random.binomial(n = 1, p = 0.5, size = 500)
y = np.random.binomial(n = 1, p = 1/(1+np.exp(X[:, 1]*2 + t*3)))
X_test = np.random.normal(size = 200).reshape((100,2))
t_test = np.random.binomial(n = 1, p = 0.5, size = 100)
y_test = np.random.binomial(n = 1, p = 1/(1+np.exp(X_test[:, 1]*2 + t_test*3)))
a = ITEEstimator(LogisticRegression(solver="lbfgs"), two_model = True)
a.fit(X, t, y)
assert type(a.model) == LogisticRegression  # test assigning a model
assert a.X_training.shape  == (500,2)       # test data passing for class
assert a.model.intercept_ is not None

In [ ]:
# a = ITEEstimator(RandomForestClassifier(), X, t, y, X_test, two_model = False )

In [ ]:
asl = ASLearner(estimator = ITEEstimator(model = RandomForestClassifier()), 
         query_strategy=random_batch_sampling,
         X_training=X,
                t_training=t,
                y_training=y,
                X_test=X_test)
asl.fit()
ite_pred, y1_pred, y0_pred = asl.predict(asl.X_test)
X_sel, query_sel = asl.query(asl.X_test, n2=10)
assert ite_pred.shape[0] == 100
assert X_sel.shape       == (10,2)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [ ]:
from xbart import XBART

In [ ]:
n_train = 100
p       = 5
n_pool  = 1000
n_test  = 1000
n2      = 20
X_train = np.random.normal(size = n_train*p).reshape((n_train,p))
t_train = np.random.binomial(n = 1, p = 0.5, size = n_train)
y_train = np.random.binomial(n = 1, p = 1/(1+np.exp(-1*(X_train[:, 1]*2 + t_train*3))))

X_pool = np.random.normal(size = n_pool*p).reshape((n_pool,p))
t_pool = np.random.binomial(n = 1, p = 0.5, size = n_pool)
y_pool = np.random.binomial(n = 1, p = 1/(1+np.exp(-1*(X_pool[:, 1]*2 + t_pool*3))))

X_test = np.random.normal(size = n_test*p).reshape((n_test,p))
t_test = np.random.binomial(n = 1, p = 0.5, size = n_test)
y_test = np.random.binomial(n = 1, p = 1/(1+np.exp(-1*(X_test[:, 1]*2 + t_test*3))))
# asl = ASLearner(estimator = ITEEstimator(model = XBART(),two_model=False), 
#          query_strategy=uncertainty_batch_sampling,
#          X_training = X_train,
#          t_training = t_train,
#          y_training = y_train,
#          X_pool     = X_pool,
#          X_test     = X_test)
# asl.fit()
# p_ite, p_y1, p_y0 = asl.predict(asl.X_test, return_mean=False)
# print("Qini before AL: {}".format(asl.score(preds=np.mean(p_ite, axis=1),
#                                             y_true=y_test, t_true=t_test)))
# qini_vals = []
# for _ in range(10):
#     X_query, ix = asl.query(asl.X_pool, n2=n2)
#     asl.teach(X_query, t_pool[ix], y_pool[ix])
#     asl.X_pool = np.delete(asl.X_pool,ix, axis=0)
#     t_pool     = np.delete(t_pool,ix, axis=0) 
#     y_pool     = np.delete(y_pool,ix, axis=0) 
#     p_ite, p_y1, p_y0 = asl.predict(asl.X_test, return_mean=False)
#     qini_vals.append(asl.score(preds=np.mean(p_ite, axis=1), y_true=y_test, t_true=t_test))
#     print("Qini after round {} of AL: {}".format(_,qini_vals[_]))

In [ ]:
asl = ASLearner(estimator = ITEEstimator(model = XBART(),two_model=False), 
         query_strategy=uncertainty_batch_sampling,
         X_training = X_train,
         t_training = t_train,
         y_training = y_train,
         X_pool     = X_pool,
         X_test     = X_test)
asl.fit()
p_ite, p_y1, p_y0 = asl.predict(asl.X_test, return_mean=False)
print("Qini before AL: {}".format(asl.score(preds=np.mean(p_ite, axis=1),
                                            y_true=y_test, t_true=t_test)))
qini_vals = []
for _ in range(10):
    X_query, ix = asl.query(asl.X_pool, n2=n2, return_mean=False)
    asl.teach(X_query, t_pool[ix], y_pool[ix])
    asl.X_pool = np.delete(asl.X_pool,ix, axis=0)
    t_pool     = np.delete(t_pool,ix, axis=0) 
    y_pool     = np.delete(y_pool,ix, axis=0) 
    p_ite, p_y1, p_y0 = asl.predict(asl.X_test, return_mean=False)
    qini_vals.append(asl.score(preds=np.mean(p_ite, axis=1), y_true=y_test, t_true=t_test))
    print("Qini after round {} of AL: {}".format(_,qini_vals[_]))

Qini before AL: 0.12159330581340701
[[0.23546293 0.39482968 0.27401979 ... 0.2147234  0.25486699 0.31302206]
 [0.21441689 0.55059787 0.37328519 ... 0.29221303 0.25333555 0.31302206]
 [0.45795969 0.55059787 0.51942893 ... 0.2147234  0.14830384 0.31302206]
 ...
 [0.33548554 0.39482968 0.37328519 ... 0.29221303 0.25333555 0.31302206]
 [0.13726011 0.53997632 0.32057036 ... 0.2147234  0.1467724  0.31302206]
 [0.45795969 0.55059787 0.50807244 ... 0.2147234  0.14830384 0.31302206]]
Qini after round 0 of AL: 0.11758657131105969
[[0.50175033 0.71411774 0.5328794  ... 0.53980752 1.08152186 0.62098956]
 [0.13575574 0.2353595  0.59732504 ... 0.49935112 0.61359864 0.84228754]
 [0.65366367 0.73234102 0.48199339 ... 0.58095258 1.22200858 0.96881801]
 ...
 [0.46400738 0.71411774 0.68356255 ... 0.53980752 0.89576561 0.94400747]
 [0.25275175 0.25358278 0.59732504 ... 0.43847564 0.74026323 0.83108558]
 [0.65366367 0.76697167 0.45409647 ... 0.58095258 0.82946696 0.96881801]]
Qini after round 1 of AL: 0.13

In [ ]:
p_ite.shape

(1000, 40)